## 순환신경망 구현 및 학습

In [1]:
import tensorflow as tf

## 하이퍼 파라미터

In [2]:
EPOCHS = 10
NUM_WORDS = 10000

## 모델 정의

In [3]:
class RNNModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        # train_data의 각 숫자는 class일뿐 숫자로서의 의미는 없다
        # 이것을 적용시켜주기 위해 embedding을 통해
        # 숫자 하나를 16개로 쪼개준다 (-> 크기자체는 커짐)
        # 1 x 32  --> 16 x 32
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        
        self.rnn = tf.keras.layers.SimpleRNN(32) # 뉴런 32개
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid') #결과는 하나.. binary라서..
        
    def __call__(self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

class LSTMModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)     
        self.lstm = tf.keras.layers.LSTM(32) # 뉴런 32개
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid') #결과는 하나.. binary라서..
        
    def __call__(self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.lstm(x)
        return self.dense(x)
    
class GRUModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        self.gru = tf.keras.layers.GRU(32) # 뉴런 32개
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid') #결과는 하나.. binary라서..
        
    def __call__(self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.gru(x)
        return self.dense(x)
    

## IMDB 데이터셋 준비
- x : 영화 평가 문장
- y : 평가가 긍정인지 부정인지 binary데이터


In [4]:
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS) ## num_words로 사용할 단어 수 제한함

C:\Users\Jang\Anaconda3\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:129: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\Jang\Anaconda3\lib\site-packages\tensorflow_core\python\keras\datasets\imdb.py:130: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [5]:
print(x_train.shape, y_train.shape)
print(len(x_train[0]), len(x_train[1])) 
print(x_train[0:3])

## 각 리스트의 맨 앞에있는 1은 SOS(start of sequence)로, 문장의 시작을 알린다
## 자주 사용하는 단어 num_words(=10000)개 만을 이용하고, 각 단어가 많이 쓰일수록 낮은 숫자로 나타낸다

(25000,) (25000,)
218 189
[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])
 list([1, 194, 1153, 194, 8

In [6]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train,
                                                        value = 0,     # padding이 필요하면 0으로 padding한다
                                                        padding='pre', # padding이 필요하면 앞으로 padding한다
                                                        maxlen=32)     # 32길이로 잘라주고, 32길이가 안되면 위의 두가지 조건 적용한다
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                        value = 0,
                                                        padding='pre',
                                                        maxlen=32)
## 문장이 너무 길다 -> 앞에 32개 단어만 가져와서 classification하겠다
x_train[0]

array([  30, 5535,   18,   51,   36,   28,  224,   92,   25,  104,    4,
        226,   65,   16,   38, 1334,   88,   12,   16,  283,    5,   16,
       4472,  113,  103,   32,   15,   16, 5345,   19,  178,   32])

In [7]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 모델 생성

In [8]:
rnn_model = RNNModel()
lstm_model = LSTMModel()
gru_model = GRUModel()

rnn_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

lstm_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

gru_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 학습 루프 동작

In [9]:
rnn_model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/10
782/782 [==============================] - 24s 31ms/step - loss: 0.5658 - accuracy: 0.6829 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
782/782 [==============================] - 18s 22ms/step - loss: 0.3723 - accuracy: 0.8358 - val_loss: 0.4862 - val_accuracy: 0.7668
Epoch 3/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2299 - accuracy: 0.9116 - val_loss: 0.5634 - val_accuracy: 0.7608
Epoch 4/10
782/782 [==============================] - 17s 22ms/step - loss: 0.1081 - accuracy: 0.9626 - val_loss: 0.7615 - val_accuracy: 0.7314
Epoch 5/10
782/782 [==============================] - 17s 21ms/step - loss: 0.0513 - accuracy: 0.9842 - val_loss: 0.9714 - val_accuracy: 0.7494
Epoch 6/10
782/782 [==============================] - 17s 21ms/step - loss: 0.0276 - accuracy: 0.9916 - val_loss: 1.1164 - val_accuracy: 0.7456
Epoch 7/10
782/782 [==============================] - 14s 18ms/step - loss: 0.0174 - accuracy: 0.9950 - val_loss: 1.2333 - val_a

In [10]:
lstm_model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/10
782/782 [==============================] - 30s 38ms/step - loss: 0.5049 - accuracy: 0.7432 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
782/782 [==============================] - 27s 34ms/step - loss: 0.3750 - accuracy: 0.8327 - val_loss: 0.4555 - val_accuracy: 0.7875
Epoch 3/10
782/782 [==============================] - 22s 28ms/step - loss: 0.3260 - accuracy: 0.8613 - val_loss: 0.4794 - val_accuracy: 0.7812
Epoch 4/10
782/782 [==============================] - 20s 26ms/step - loss: 0.2843 - accuracy: 0.8836 - val_loss: 0.4990 - val_accuracy: 0.7797
Epoch 5/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2342 - accuracy: 0.9077 - val_loss: 0.6789 - val_accuracy: 0.7631
Epoch 6/10
782/782 [==============================] - 26s 33ms/step - loss: 0.1829 - accuracy: 0.9296 - val_loss: 0.6970 - val_accuracy: 0.7592
Epoch 7/10
782/782 [==============================] - 24s 31ms/step - loss: 0.1395 - accuracy: 0.9513 - val_loss: 0.8057 - val_a

In [11]:
gru_model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/10
782/782 [==============================] - 29s 37ms/step - loss: 0.5078 - accuracy: 0.7373 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/10
782/782 [==============================] - 25s 32ms/step - loss: 0.3747 - accuracy: 0.8316 - val_loss: 0.4626 - val_accuracy: 0.7817
Epoch 3/10
782/782 [==============================] - 25s 32ms/step - loss: 0.3269 - accuracy: 0.8587 - val_loss: 0.4832 - val_accuracy: 0.7792
Epoch 4/10
782/782 [==============================] - 25s 32ms/step - loss: 0.2867 - accuracy: 0.8828 - val_loss: 0.5128 - val_accuracy: 0.7802
Epoch 5/10
782/782 [==============================] - 26s 33ms/step - loss: 0.2425 - accuracy: 0.9055 - val_loss: 0.5592 - val_accuracy: 0.7710
Epoch 6/10
782/782 [==============================] - 25s 32ms/step - loss: 0.1998 - accuracy: 0.9252 - val_loss: 0.6890 - val_accuracy: 0.7577
Epoch 7/10
782/782 [==============================] - 24s 31ms/step - loss: 0.1612 - accuracy: 0.9445 - val_loss: 0.7064 - val_a